## Problem Description

I have, in the below project worked on IMDB movie review sentiment classification.Each movie review is a variable sequence of words and the sentiment of each movie review must be classified.

The Large Movie Review Dataset (often referred to as the IMDB dataset) contains 25,000  movie reviews (good or bad) for training and the same amount again for testing. The problem is to determine whether a given movie review has a positive or negative sentiment.



Keras provides access to the IMDB dataset built-in. <font color='green'>The *imdb.load_data()* function allows you to load the dataset in a format that is ready for use in neural network and deep learning models.</font>

The words have been replaced by integers that indicate the ordered frequency of each word in the dataset. The sentences in each review are therefore comprised of a sequence of integers.

### Word Embedding
1. We will map each movie review as a vector, a popular technique when working with text called word embedding. This is a technique where words are encoded as real-valued vectors in a high dimensional space, where the similarity between words in terms of meaning translates to closeness in the vector space.

Keras provides a convenient way to convert positive integer representations of words into a word embedding by an **Embedding** layer.

2. We will map each word onto a 32 length real valued vector. We will also limit the total number of words that we are interested in modeling to the 5000 most frequent words, and zero out the rest. Finally, the sequence length (number of words) in each review varies, so we will constrain each review to be 500 words, truncating long reviews and pad the shorter reviews with zero values.

Now that we have defined how the data will be prepared and modeled, we are ready to develop an LSTM model to classify the sentiment of movie reviews.

### Simple LSTM for Sequence Classification
I have developed a small LSTM for the IMDB problem and achieve good accuracy.


In [6]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

We need to load the IMDB dataset. We are constraining the dataset to the top 5,000 words. We also split the dataset into train (50%) and test (50%) sets.

In [ ]:
''' Actual code is this-'''
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# load the dataset but only keep the top n words, zero the rest
# But because for some reasons this couldn't be loaded, we use the code below as a trick to load the same dataset

In [8]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

# restore np.load for future normal usage
np.load = np_load_old

 In sentiment classification Keras prefers to have all inputs of same dimension/ length.
So, i have truncated and padded the input sequences so that they are all the same length for modeling.

In [9]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

## Model -

Now define, compile and fit our LSTM model.

* The first layer is the Embedded layer that uses 32 length vectors to represent each word. The next layer is the LSTM layer with 100 memory units (smart neurons).

* Finally, because this is a classification problem we use a Dense output layer with a single neuron and a sigmoid activation function to make 0 or 1 predictions for the two classes (good and bad) in the problem.

* Because it is a binary classification problem, log loss is used as the loss function (*binary_crossentropy* in Keras). The efficient ADAM optimization algorithm is used. The model is fit for only 2 epochs because it quickly overfits the problem. A large batch size of 64 reviews is used to space out weight updates.

In [10]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# Embedding converts positive integers to a vector of fixed size (given by 2nd arg)
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 182s 7ms/step - loss: 0.4365 - acc: 0.7937 - val_loss: 0.3252 - val_acc: 0.8651
Epoch 2/3
25000/25000 [==============================] - 188s 8ms/step - loss: 0.3642 - acc: 0.8405 - val_loss: 0.5864 - val_acc: 0.6738
Epoch 3/3
25000/25000 [==============================] - 181s 7ms/step 

## Testing the model
Once fit, we estimate the performance of the model on unseen reviews

In [11]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.25%


### LSTM For Sequence Classification With Dropout
Recurrent Neural networks like LSTM generally have the problem of overfitting.

Dropout can be applied between layers using the Dropout Keras layer. We can do this easily by adding new Dropout layers between the Embedding and LSTM layers and the LSTM and Dense output layers. I leave that for u to try :P





We have achieved a good accuracy (actually a decent one,but never mind) by using a simple LSTM model (that too without dropout). Even better accuracy can be achieved by using Dropout, regularizations and using 1D convolutions alongside LSTM.